In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Welcome to the Sample python script to read Dicom file and convert into jpg format. Dataset is being used from [SIIM-FISABIO-RSNA COVID-19 Detection](https://www.kaggle.com/c/siim-covid19-detection/data) competition.
![](https://storage.googleapis.com/kaggle-competitions/kaggle/26680/logos/header.png)

In [ ]:
import pydicom as dicom
from PIL import Image
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os
import ast
from colorama import Style

Display list of content of directory siim-covid19-detection

In [ ]:
path = '/kaggle/input/siim-covid19-detection/'
os.listdir(path)

# Load Data

In [ ]:
train_image = pd.read_csv(path+'train_image_level.csv')

In [ ]:
train_image.head().style.applymap(lambda x: 'background-color:lightgreen')

Displaying content of the folder of the first row in train_image

In [ ]:
for dirname, _, filenames in os.walk(path+'/train/5776db0cec75'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
path_train = path+'train/'+train_image.loc[0, 'StudyInstanceUID']+'/'+'81456c9c5423'+'/'
print(path_train)

In [ ]:
image_id = train_image.loc[0, 'id']
print(image_id)

Read value of column "id" and replace _image with .dcm extension

In [ ]:
img_id = train_image.loc[0, 'id'].replace('_image', '.dcm')
print(img_id)

Read Dicom file and Display image

In [ ]:
data_file = dicom.dcmread(path_train+img_id)
# Extract image data of the dicom file
img = data_file.pixel_array
fig, ax = plt.subplots(1, 1, figsize=(16, 8))
ax.imshow(img)

Save file into Output folder with name as "image_id".jpg

In [ ]:
im = data_file.pixel_array.astype(float)
rescaled_image = (np.maximum(im,0)/im.max())*255 # float pixels
final_image = np.uint8(rescaled_image) # integers pixels
final_image = Image.fromarray(final_image)
final_image.save('./'+image_id+'.jpg')


Display required number of Dicom Images and save them as jpg format

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(20, 20))
fig.subplots_adjust(hspace = .1, wspace=.1)
axs = axs.ravel()

for row in range(9):
    study = train_image.loc[row, 'StudyInstanceUID']
    path_in = path+'train/'+study+'/'
    folder = os.listdir(path_in)
    path_file = path_in+folder[0]
    filename = os.listdir(path_file)[0]
    file_id = filename.split('.')[0]
    
    data_file = dicom.dcmread(path_file+'/'+file_id+'.dcm')
    img = data_file.pixel_array
    if (train_image.loc[row, 'boxes']!=train_image.loc[row, 'boxes']) == False:
        boxes = ast.literal_eval(train_image.loc[row, 'boxes'])
    
        for box in boxes:
            p = matplotlib.patches.Rectangle((box['x'], box['y']), box['width'], box['height'],
                                     ec='r', fc='none', lw=2.)
            axs[row].add_patch(p)
    axs[row].imshow(img, cmap='gray')
    axs[row].set_title(train_image.loc[row, 'label'].split(' ')[0])
    axs[row].set_xticklabels([])
    axs[row].set_yticklabels([])
    im = data_file.pixel_array.astype(float)
    rescaled_image = (np.maximum(im,0)/im.max())*255 # float pixels
    final_image = np.uint8(rescaled_image) # integers pixels
    final_image = Image.fromarray(final_image)
    final_image.save('./'+file_id+'.jpg')